In [1]:
import os 
import tensorflow as tf

2025-01-27 19:45:43.233241: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-27 19:45:43.293126: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-27 19:45:44.661137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
%cd /home/lab/Workspace/e2e-mlflow-dvc

/home/lab/Workspace/e2e-mlflow-dvc


In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/diazmanne/e2e-mlflow-dvc.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="diazmanne"
os.environ["MLFLOW_TRACKING_PASSWORD"]="f4e87115c3d98a2ef151c0469443be9738fa35ff"

In [4]:
model = tf.keras.models.load_model("artifacts/training/trained_model.keras")

In [5]:
from dataclasses import dataclass
from pathlib import Path
from typing import List, Dict



@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    test_data: Path  # Added test_data here to match the code usage
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

# Validation function
def validate_evaluation_config(evaluation_config: EvaluationConfig):
    print(f"Model Path: {evaluation_config.path_of_model}")
    print(f"Training Data Path: {evaluation_config.training_data}")
    print(f"Test Data Path: {evaluation_config.test_data}")
    print(f"MLflow URI: {evaluation_config.mlflow_uri}")
    print(f"Image Size: {evaluation_config.params_image_size}")
    print(f"Batch Size: {evaluation_config.params_batch_size}")
    print(f"All Params: {evaluation_config.all_params}")

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Update paths to match actual data location
        self.config["evaluation"]["training_data_path"] = "artifacts/data_ingestion/Chest-CT-Scan-data"
        self.config["evaluation"]["test_data_path"] = "artifacts/data_ingestion/Chest-CT-Scan-data"
        
        create_directories([self.config["artifacts_root"]])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path(self.config["evaluation"]["trained_model_path"]),
            training_data=Path(self.config["evaluation"]["training_data_path"]),
            test_data=Path(self.config["evaluation"]["test_data_path"]),
            mlflow_uri=self.config["mlflow_uri"],
            all_params=self.params,
            params_image_size=self.params["IMAGE_SIZE"],
            params_batch_size=self.params["BATCH_SIZE"]
        )
        return eval_config

# Validation function
def validate_evaluation_config(evaluation_config: EvaluationConfig):
    print(f"Model Path: {evaluation_config.path_of_model}")
    print(f"Training Data Path: {evaluation_config.training_data}")
    print(f"Test Data Path: {evaluation_config.test_data}")
    print(f"MLflow URI: {evaluation_config.mlflow_uri}")
    print(f"Image Size: {evaluation_config.params_image_size}")
    print(f"Batch Size: {evaluation_config.params_batch_size}")
    print(f"All Params: {evaluation_config.all_params}")

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import logging
from cnnClassifier.utils.common import save_json
import mlflow.tensorflow  

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None
        self.valid_generator = None
        self.score = None
        logging.info("Evaluation instance created successfully.")

    def _valid_generator(self):  # Fixed method name (removed asterisks)
        try:
            logging.info("Initializing the validation data generator.")
            
            datagenerator_kwargs = dict(
                rescale=1.0 / 255,
                validation_split=0.30
            )
            
            dataflow_kwargs = dict(
                target_size=self.config.params_image_size[:-1],
                batch_size=self.config.params_batch_size,
                interpolation="bilinear",
                class_mode='categorical'  # Added class_mode for classification
            )
            
            valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )
            
            self.valid_generator = valid_datagenerator.flow_from_directory(
                directory=str(self.config.training_data),  # Convert Path to string
                subset="validation",
                shuffle=False,
                **dataflow_kwargs
            )
            logging.info("Validation data generator initialized successfully.")
            
        except Exception as e:
            logging.error(f"Error initializing validation generator: {e}")
            raise e

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        try:
            logging.info(f"Loading model from: {path}")
            model = tf.keras.models.load_model(str(path))  # Convert Path to string
            logging.info("Model loaded successfully.")
            return model
        except Exception as e:
            logging.error(f"Error loading model: {e}")
            raise e

    def evaluation(self):
        try:
            logging.info("Starting model evaluation.")
            self.model = self.load_model(self.config.path_of_model)
            self._valid_generator()
            
            if self.valid_generator is None:
                raise ValueError("Validation generator not initialized.")
            
            self.score = self.model.evaluate(
                self.valid_generator,
                steps=len(self.valid_generator)  # Added steps parameter
            )
            logging.info(f"Evaluation completed. Loss: {self.score[0]}, Accuracy: {self.score[1]}")
            self.save_score()
            
        except Exception as e:
            logging.error(f"Error during evaluation: {e}")
            raise e

    def save_score(self):
        try:
            logging.info("Saving evaluation scores.")
            scores = {"loss": float(self.score[0]), "accuracy": float(self.score[1])}  # Convert to Python float
            save_json(path=Path("scores.json"), data=scores)
            logging.info(f"Scores saved successfully: {scores}")
        except Exception as e:
            logging.error(f"Error saving scores: {e}")
            raise e

    def log_into_mlflow(self):
        try:
            logging.info("Logging metrics and model to MLflow.")
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            
            with mlflow.start_run():
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics({
                    "loss": float(self.score[0]),
                    "accuracy": float(self.score[1])
                })
                
                # Log model using mlflow.tensorflow
                logging.info("Generating model signature for MLflow logging.")
                input_data = next(iter(self.valid_generator))[0][:5]  # Get sample inputs
                predictions = self.model.predict(input_data)
                
                # Highlighted code
                mlflow.tensorflow.log_model(
                    self.model,
                    "model",
                    signature=mlflow.models.signature.infer_signature(input_data, predictions)
                )
                logging.info("Model logged to MLflow successfully.")
                print(tf.__version__)  # Instead of tensorflow.keras.__version__
                
        except Exception as e:
            logging.error(f"Error during MLflow logging: {e}")
            raise e

In [ ]:
try:
    # Initialize configuration
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    
    # Validate configuration
    validate_evaluation_config(eval_config)
    
    # Perform evaluation
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    
    # Log results to MLflow
    evaluation.log_into_mlflow()
    
except FileNotFoundError as e:
    print(f"File not found error: {str(e)}")
    print("Please check if all the paths in the configuration are correct.")
    raise e
except Exception as e:
    print(f"An error occurred: {type(e).__name__}")
    print(f"Error message: {str(e)}")
    raise e

[2025-01-27 19:45:49,577: INFO: common: YAML file loaded successfully: config/config.yaml]
[2025-01-27 19:45:49,581: INFO: common: YAML file loaded successfully: params.yaml]
[2025-01-27 19:45:49,583: INFO: common: Created directory at: artifacts]
Model Path: artifacts/training/trained_model.keras
Training Data Path: artifacts/data_ingestion/Chest-CT-Scan-data
Test Data Path: artifacts/data_ingestion/Chest-CT-Scan-data
MLflow URI: https:/dagshub.com/diazmanne/e2e-mlflow-dvc.mlflow
Image Size: [224, 224, 3]
Batch Size: 16
All Params: {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': PosixPath('imagenet'), 'LEARNING_RATE': 0.01}
[2025-01-27 19:45:49,584: INFO: 4223989291: Evaluation instance created successfully.]
[2025-01-27 19:45:49,585: INFO: 4223989291: Starting model evaluation.]
[2025-01-27 19:45:49,587: INFO: 4223989291: Loading model from: artifacts/training/trained_model.keras]
[2025-01-27 19:45:51,8